In [1]:
from crawl import Crawler
from parse import find_all_safely
import pandas as pd
import os
from setup_db import DB, setup_db, cursor_as_df, execute_to_df

In [2]:
db = setup_db()

In [3]:
# db.close()

In [4]:
db.select_all("jobs")

,job_id,user_created,active,time_created,last_run,job_name
0,1,1,1,2019-10-10 00:00:00,None,test_job


In [5]:
db.select_all("requests")

,request_id,job_id,start_city,end_city,date
0,1,1,1915,1815,2020-01-02 00:00:00
1,2,1,1915,7588,2020-01-02 00:00:00
2,3,1,1915,7588,2020-01-03 00:00:00


SELECT 
strftime('%d.%m.%Y',t2.time_created) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1


In [6]:
params_from_db = db.c.execute('''SELECT 
strftime("%d.%m.%Y",t2.date) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests as t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1''')
colnames = [i[0] for i in params_from_db.description]
values = params_from_db.fetchall()

In [7]:
print(colnames)
print(values)

['rideDate', 'departureCity', 'arrivalCity']
[('02.01.2020', 1915, 1815), ('02.01.2020', 1915, 7588), ('03.01.2020', 1915, 7588)]


In [8]:
constant_search_params = {
    "adult": "1",
    "_locale": "pl"
}


In [9]:
params_list = []
for single_params in values:
    params_list.append({"rideDate": single_params[0],
                       "departureCity": single_params[1],
                       "arrivalCity": single_params[2]})
    
params_list = [{**i, **constant_search_params} for i in params_list]

In [10]:
params_list

[{'_locale': 'pl',
  'adult': '1',
  'arrivalCity': 1815,
  'departureCity': 1915,
  'rideDate': '02.01.2020'},
 {'_locale': 'pl',
  'adult': '1',
  'arrivalCity': 7588,
  'departureCity': 1915,
  'rideDate': '02.01.2020'},
 {'_locale': 'pl',
  'adult': '1',
  'arrivalCity': 7588,
  'departureCity': 1915,
  'rideDate': '03.01.2020'}]

In [12]:
crawler = Crawler(params_list[1])
crawler.get_all()
crawler.get_pandas_df()

,arrival_station,arrival_time,departure_station,departure_time,price
0,Łódź Fabryczna,04:00,Kraków MDA,00:30,"59,99"
1,Łódź Fabryczna,11:25,Kraków MDA,07:05,"59,99"
2,Łódź Fabryczna,16:00,Kraków MDA,12:00,"69,99"
3,Łódź Fabryczna,19:25,Kraków MDA,15:10,"69,99"
4,Łódź Fabryczna,23:50,Kraków MDA,19:15,"69,99"
5,Łódź Fabryczna,04:15,Kraków MDA,23:59,"49,99"
6,Łódź Fabryczna,04:00,Kraków MDA,00:30,"69,99"


In [23]:
crawler._request.url

'https://shop.flixbus.pl/search?'

In [36]:
crawler._request._resp.url

'https://shop.flixbus.pl/search?arrivalCity=7588&_locale=pl&departureCity=1915&rideDate=12.12.2019&adult=1'

In [13]:
results = []
for search_params in params_list:
    crawler = Crawler(search_params)
    crawler.get_all()
    results.append(crawler.get_pandas_df())
    
results

[      arrival_station arrival_time departure_station departure_time   price
 0              Gdańsk        09:35        Kraków MDA          00:30  109,99
 1  Gdańsk Hala Olivia        11:45        Kraków MDA          00:30  105,98,
   arrival_station arrival_time departure_station departure_time  price
 0  Łódź Fabryczna        04:00        Kraków MDA          00:30  59,99
 1  Łódź Fabryczna        11:25        Kraków MDA          07:05  59,99
 2  Łódź Fabryczna        16:00        Kraków MDA          12:00  69,99
 3  Łódź Fabryczna        19:25        Kraków MDA          15:10  69,99
 4  Łódź Fabryczna        23:50        Kraków MDA          19:15  69,99
 5  Łódź Fabryczna        04:15        Kraków MDA          23:59  49,99
 6  Łódź Fabryczna        04:00        Kraków MDA          00:30  69,99,
   arrival_station arrival_time departure_station departure_time  price
 0  Łódź Fabryczna        04:00        Kraków MDA          00:30  69,99
 1  Łódź Fabryczna        11:25        Kraków M

Crawler

In [ ]:
for i in params.itertuples():
    a = i

In [ ]:
a.departureCity

In [27]:
variable_search_params = {
    "departureCity": "7568",
    "arrivalCity": "1915",
    "rideDate": "22.12.2019"
}

In [28]:
search_params = {
    **variable_search_params, **constant_search_params
}

In [29]:
crawler = Crawler(search_params)
crawler.get_all()

In [34]:
crawler._request._resp.url

'https://shop.flixbus.pl/search?arrivalCity=1915&_locale=pl&departureCity=7568&rideDate=22.12.2019&adult=1'

In [30]:
crawler.get_pandas_df()

,arrival_station,arrival_time,departure_station,departure_time,price
0,Kraków MDA,11:00,Warszawa Zachodnia,06:35,"59,99"
1,Kraków MDA,11:55,Warszawa Wilanowska,07:00,"85,99"
2,Kraków MDA,13:25,Warszawa Zachodnia,09:00,"59,99"
3,Kraków MDA,14:55,Warszawa Wilanowska,10:00,"85,99"
4,Kraków MDA,16:20,Warszawa Zachodnia,12:00,"79,99"
5,Kraków MDA,19:25,Warszawa Wilanowska,14:30,"75,99"
6,Kraków MDA,19:55,Warszawa Zachodnia,15:30,"69,99"
7,Kraków MDA,20:45,Warszawa Zachodnia,16:20,"49,99"
8,Kraków MDA,22:10,Warszawa Zachodnia,17:45,"29,99"
9,Kraków MDA,22:55,Warszawa Zachodnia,18:30,"59,99"
